In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.cluster import KMeans

import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

from math import sqrt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import joblib


In [2]:
cna_df = pd.read_csv("./trainData/train_data_cna.csv", encoding='utf-8')

In [3]:
cna_df

,Hugo_Symbol,Entrez_Gene_Id,TCGA-3M-AB46-01,TCGA-3M-AB47-01,TCGA-B7-5816-01,TCGA-B7-5818-01,TCGA-B7-A5TI-01,TCGA-B7-A5TJ-01,TCGA-B7-A5TK-01,TCGA-B7-A5TN-01,...,TCGA-VW-A8FI-01,TCGA-W9-A837-01,TCGA-WH-A86K-01,TCGA-WY-A858-01,TCGA-WY-A859-01,TCGA-WY-A85A-01,TCGA-WY-A85B-01,TCGA-WY-A85C-01,TCGA-WY-A85D-01,TCGA-WY-A85E-01
0,ACAP3,116983.0,0.362,0.000,0.030,0.022,0.019,-0.294,-0.005,-0.402,...,0.030,-0.785,0.014,0.005,-0.020,-0.008,-0.028,0.015,0.021,0.052
1,ACTRT2,140625.0,0.362,0.000,0.030,0.022,0.019,-0.294,-0.005,-0.402,...,0.030,-0.785,0.014,0.005,-0.020,-0.008,-0.028,0.015,0.021,0.052
2,AGRN,375790.0,0.362,0.000,0.030,0.022,0.019,-0.294,-0.005,-0.402,...,0.030,-0.785,0.014,0.005,-0.020,-0.008,-0.028,0.015,0.021,0.052
3,ANKRD65,441869.0,0.362,0.000,0.030,0.022,0.019,-0.294,-0.005,-0.402,...,0.030,-0.785,0.014,0.005,-0.020,-0.008,-0.028,0.015,0.021,0.052
4,ATAD3A,55210.0,0.362,0.000,0.030,0.022,0.019,-0.294,-0.005,-0.402,...,0.030,-0.785,0.014,0.005,-0.020,-0.008,-0.028,0.015,0.021,0.052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25123,VBP1,7411.0,-0.188,0.321,0.054,1.050,-0.169,0.204,0.015,1.020,...,0.018,-0.067,-0.045,0.203,-0.385,0.006,0.164,-0.028,-0.065,-0.598
25124,DDX11L16|chrX,727856.0,-0.188,0.321,0.054,1.050,-0.169,0.204,0.015,1.020,...,0.018,-0.067,-0.045,0.203,-0.385,0.006,0.164,-0.028,-0.065,-0.598
25125,IL9R|chrX,3581.0,-0.188,0.321,0.054,1.050,-0.169,0.204,0.015,1.020,...,0.018,-0.067,-0.045,0.203,-0.385,0.006,0.164,-0.028,-0.065,-0.598
25126,SPRY3|chrX,10251.0,-0.188,0.321,0.054,1.050,-0.169,0.204,0.015,1.020,...,0.018,-0.067,-0.045,0.203,-0.385,0.006,0.164,-0.028,-0.065,-0.598


In [4]:
patient_df = pd.read_csv("./trainData/train_data_clinical_patient.csv", encoding='utf-8')

In [5]:
patient_df

,PATIENT_ID,CANCER_TYPE_ACRONYM,OTHER_PATIENT_ID,AGE,SEX,AJCC_PATHOLOGIC_TUMOR_STAGE,AJCC_STAGING_EDITION,DAYS_LAST_FOLLOWUP,DAYS_TO_BIRTH,DAYS_TO_INITIAL_PATHOLOGIC_DIAGNOSIS,...,CENTER,IN_PANCANPATHWAYS_FREEZE,OS_STATUS,OS_MONTHS,DSS_STATUS,DSS_MONTHS,DFS_STATUS,DFS_MONTHS,PFS_STATUS,PFS_MONTHS
0,TCGA-3M-AB46,STAD,BE6531B2-D1F3-44AB-9C02-1CEAE51EF2BB,70.0,Male,STAGE IB,6TH,1765,NaN,0.0,...,University of Kansas Medical Center,Yes,0:LIVING,58.026761,0:ALIVE OR DEAD TUMOR FREE,58.026761,0:DiseaseFree,58.026761,0:CENSORED,58.026761
1,TCGA-3M-AB47,STAD,85C11B74-9E50-4DA1-8C0B-D5677CC801B1,51.0,Male,STAGE IIIB,6TH,NaN,NaN,0.0,...,University of Kansas Medical Center,Yes,1:DECEASED,NaN,1:DEAD WITH TUMOR,NaN,NaN,NaN,1:PROGRESSION,12.986159
2,TCGA-B7-5816,STAD,f07070c0-fd0a-4c19-ba1e-5f06b933cd7c,51.0,Female,STAGE IIB,7TH,812,-18698,0.0,...,Cureline,Yes,0:LIVING,26.695598,0:ALIVE OR DEAD TUMOR FREE,26.695598,0:DiseaseFree,26.695598,0:CENSORED,26.695598
3,TCGA-B7-5818,STAD,6e03b415-84a1-4b91-8717-1a41edd4a255,62.0,Male,STAGE IB,7TH,356,-22792,0.0,...,Cureline,Yes,0:LIVING,11.703981,0:ALIVE OR DEAD TUMOR FREE,11.703981,NaN,NaN,0:CENSORED,11.703981
4,TCGA-B7-A5TI,STAD,4310A287-5F01-4E0D-94E3-96C5379C3245,52.0,Male,STAGE IIIC,7TH,595,-19014,0.0,...,Cureline,Yes,0:LIVING,19.561429,0:ALIVE OR DEAD TUMOR FREE,19.561429,0:DiseaseFree,19.561429,0:CENSORED,19.561429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692,TCGA-WY-A85B,LGG,CB041720-0EFA-4EB6-A9A5-B6E138BB1A99,24.0,Male,NaN,NaN,1393,-8817,0.0,...,Johns Hopkins,Yes,0:LIVING,45.796758,0:ALIVE OR DEAD TUMOR FREE,45.796758,0:DiseaseFree,45.796758,0:CENSORED,45.796758
2693,TCGA-WY-A85C,LGG,F62BE2B8-88C4-4A7E-8C80-8176FB5F6C56,36.0,Male,NaN,NaN,1426,-13506,0.0,...,Johns Hopkins,Yes,0:LIVING,46.881678,0:ALIVE OR DEAD TUMOR FREE,46.881678,NaN,NaN,1:PROGRESSION,26.596969
2694,TCGA-WY-A85D,LGG,0D5A4C63-C3FB-4295-A433-D90CDBFC4ED6,60.0,Male,NaN,NaN,1147,-21979,0.0,...,Johns Hopkins,Yes,0:LIVING,37.709176,0:ALIVE OR DEAD TUMOR FREE,37.709176,NaN,NaN,1:PROGRESSION,29.161324
2695,TCGA-WY-A85E,LGG,2272DFF7-A509-496F-B500-467C1BCE2F79,48.0,Female,NaN,NaN,633,-17878,0.0,...,Johns Hopkins,Yes,0:LIVING,20.810731,0:ALIVE OR DEAD TUMOR FREE,20.810731,NaN,NaN,1:PROGRESSION,17.490219


In [6]:
clinical = pd.read_csv('./trainData/train_data_clinical_patient.csv', usecols=['PATIENT_ID', 'CANCER_TYPE_ACRONYM'])

In [7]:
clinical

,PATIENT_ID,CANCER_TYPE_ACRONYM
0,TCGA-3M-AB46,STAD
1,TCGA-3M-AB47,STAD
2,TCGA-B7-5816,STAD
3,TCGA-B7-5818,STAD
4,TCGA-B7-A5TI,STAD
...,...,...
2692,TCGA-WY-A85B,LGG
2693,TCGA-WY-A85C,LGG
2694,TCGA-WY-A85D,LGG
2695,TCGA-WY-A85E,LGG


In [9]:
from sklearn.linear_model import LogisticRegression

In [14]:
cna_dfset = cna_df.drop(['Hugo_Symbol'],axis=1)

cna_dfset=cna_dfset.transpose()

cna_dfset = cna_dfset.rename(columns=cna_dfset.iloc[0])

cna_dfset.index = cna_dfset.index.str.rstrip('-01')

cna_dfset = cna_dfset.drop(cna_dfset.index[0])
cna_dfset

,116983.0,140625.0,375790.0,441869.0,55210.0,83858.0,219293.0,54998.0,126792.0,54991.0,...,101927830.0,100132963.0,677817.0,677835.0,55217.0,7411.0,727856.0,3581.0,10251.0,6845.0
TCGA-3M-AB46,0.362,0.362,0.362,0.362,0.362,0.362,0.362,0.362,0.362,0.362,...,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188
TCGA-3M-AB47,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321
TCGA-B7-5816,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.030,...,0.054,0.054,0.054,0.054,0.054,0.054,0.054,0.054,0.054,0.054
TCGA-B7-5818,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.022,...,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050
TCGA-B7-A5TI,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,...,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WY-A85A,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,...,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006
TCGA-WY-A85B,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,...,0.157,0.157,0.157,0.157,0.157,0.164,0.164,0.164,0.164,0.164
TCGA-WY-A85C,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,...,0.011,0.011,0.011,0.011,-0.019,-0.028,-0.028,-0.028,-0.028,-0.028
TCGA-WY-A85D,0.021,0.021,0.021,0.021,0.021,0.021,0.021,0.021,0.021,0.021,...,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065


In [173]:
cna_dfset.index[1].strip()

'TCGA-3M-AB47'

In [15]:
# cna_dfset 데이터프레임과 clinical 데이터프레임에서 필요한 열 선택

# cna_dfset에 Cancer 열 추가
cna_dfset['Cancer'] = ''

# clinical 데이터프레임을 순회하며 cna_dfset에 값 할당
for index, row in cna_dfset.iterrows():
    for i, pat in clinical.iterrows():
        if(index==pat['PATIENT_ID']):
            cancer_type = pat['CANCER_TYPE_ACRONYM']
            cna_dfset.at[index, 'Cancer'] = cancer_type
            break
cna_dfset

,116983.0,140625.0,375790.0,441869.0,55210.0,83858.0,219293.0,54998.0,126792.0,54991.0,...,100132963.0,677817.0,677835.0,55217.0,7411.0,727856.0,3581.0,10251.0,6845.0,Cancer
TCGA-3M-AB46,0.362,0.362,0.362,0.362,0.362,0.362,0.362,0.362,0.362,0.362,...,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,STAD
TCGA-3M-AB47,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,STAD
TCGA-B7-5816,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.030,...,0.054,0.054,0.054,0.054,0.054,0.054,0.054,0.054,0.054,STAD
TCGA-B7-5818,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.022,...,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050,STAD
TCGA-B7-A5TI,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,...,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,STAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WY-A85A,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,...,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,LGG
TCGA-WY-A85B,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,...,0.157,0.157,0.157,0.157,0.164,0.164,0.164,0.164,0.164,LGG
TCGA-WY-A85C,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,...,0.011,0.011,0.011,-0.019,-0.028,-0.028,-0.028,-0.028,-0.028,LGG
TCGA-WY-A85D,0.021,0.021,0.021,0.021,0.021,0.021,0.021,0.021,0.021,0.021,...,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,LGG


In [266]:
cna_dfready = cna_dfset.dropna(subset=['Cancer'])
cna_dfready

In [16]:
cna_dfready = cna_dfset[cna_dfset['Cancer'] != '']
cna_dfready

,116983.0,140625.0,375790.0,441869.0,55210.0,83858.0,219293.0,54998.0,126792.0,54991.0,...,100132963.0,677817.0,677835.0,55217.0,7411.0,727856.0,3581.0,10251.0,6845.0,Cancer
TCGA-3M-AB46,0.362,0.362,0.362,0.362,0.362,0.362,0.362,0.362,0.362,0.362,...,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,-0.188,STAD
TCGA-3M-AB47,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,0.321,STAD
TCGA-B7-5816,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.030,...,0.054,0.054,0.054,0.054,0.054,0.054,0.054,0.054,0.054,STAD
TCGA-B7-5818,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.022,...,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050,1.050,STAD
TCGA-B7-A5TI,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,...,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,-0.169,STAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WY-A85A,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,...,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,LGG
TCGA-WY-A85B,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,-0.028,...,0.157,0.157,0.157,0.157,0.164,0.164,0.164,0.164,0.164,LGG
TCGA-WY-A85C,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,...,0.011,0.011,0.011,-0.019,-0.028,-0.028,-0.028,-0.028,-0.028,LGG
TCGA-WY-A85D,0.021,0.021,0.021,0.021,0.021,0.021,0.021,0.021,0.021,0.021,...,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,LGG


In [280]:
cna_dfready.tail(40)

,116983.0,140625.0,375790.0,441869.0,55210.0,83858.0,219293.0,54998.0,126792.0,54991.0,...,100132963.0,677817.0,677835.0,55217.0,7411.0,727856.0,3581.0,10251.0,6845.0,Cancer
TCGA-TM-A84S,-0.777,-0.777,-0.777,-0.777,-0.777,-0.777,-0.777,-0.777,-0.777,-0.777,...,-0.083,-0.083,-0.083,-0.083,-0.083,-0.083,-0.083,-0.083,-0.083,LGG
TCGA-TM-A84T,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,0.015,LGG
TCGA-TQ-A7RF,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,...,-0.067,-0.067,-0.067,-0.067,-0.140,-0.140,-0.140,-0.140,-0.140,LGG
TCGA-TQ-A7RG,-0.794,-0.794,-0.794,-0.794,-0.794,-0.794,-0.794,-0.794,-0.794,-0.794,...,-0.048,-0.048,-0.048,-0.048,-0.048,-0.048,-0.048,-0.048,-0.048,LGG
TCGA-TQ-A7RH,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,...,-0.023,-0.023,-0.023,-0.023,-0.023,-0.023,-0.023,-0.023,-0.023,LGG
TCGA-TQ-A7RI,-0.779,-0.779,-0.779,-0.779,-0.779,-0.779,-0.779,-0.779,-0.779,-0.779,...,-0.038,-0.038,-0.038,-0.038,-0.038,-0.038,-0.038,-0.038,-0.038,LGG
TCGA-TQ-A7RJ,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,...,0.118,0.118,0.118,0.118,0.118,0.118,0.118,0.118,0.118,LGG
TCGA-TQ-A7RK,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,...,0.179,0.179,0.179,0.191,0.191,0.191,0.191,0.191,0.191,LGG
TCGA-TQ-A7RM,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,-0.941,-0.043,-0.043,-0.043,-0.043,-0.043,-0.043,-0.043,-0.043,LGG
TCGA-TQ-A7RN,-0.762,-0.762,-0.762,-0.762,-0.762,-0.762,-0.762,-0.762,-0.762,-0.762,...,-0.046,-0.046,-0.046,-0.046,-0.046,-0.046,-0.046,-0.046,-0.046,LGG


In [17]:
cna_dfready.isnull().sum()

116983.0    0
140625.0    0
375790.0    0
441869.0    0
55210.0     0
           ..
727856.0    0
3581.0      0
10251.0     0
6845.0      0
Cancer      0
Length: 25129, dtype: int64

In [18]:
X = cna_dfready.drop("Cancer", axis=1)
y = cna_dfready["Cancer"]

In [19]:
from sklearn.impute import SimpleImputer

# 결측값을 평균으로 대체하는 SimpleImputer 생성
imputer = SimpleImputer(strategy='mean')

# X 데이터에 대해 결측값 대체 수행
X = imputer.fit_transform(X)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

LogisticRegression(max_iter=500)

In [29]:
final_model = LogisticRegression(max_iter=500)
final_model.fit(X, y)

LogisticRegression(max_iter=500)

In [22]:
y_pred = model.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8407310704960835


In [288]:
y_pred

array(['STAD', 'LUAD', 'LIHC', 'LGG', 'LUAD', 'LGG', 'STAD', 'LGG',
       'STAD', 'LUAD', 'LIHC', 'STAD', 'LUAD', 'LGG', 'LUAD', 'COAD',
       'LGG', 'LGG', 'STAD', 'LIHC', 'STAD', 'LGG', 'STAD', 'STAD', 'LGG',
       'LIHC', 'STAD', 'COAD', 'COAD', 'STAD', 'LGG', 'STAD', 'LGG',
       'LIHC', 'COAD', 'LIHC', 'COAD', 'LUAD', 'LGG', 'LGG', 'LIHC',
       'LUAD', 'LUAD', 'COAD', 'LUAD', 'LUAD', 'COAD', 'COAD', 'STAD',
       'COAD', 'LIHC', 'LUAD', 'COAD', 'LIHC', 'STAD', 'LIHC', 'LGG',
       'STAD', 'LGG', 'LGG', 'STAD', 'LUAD', 'LUAD', 'LIHC', 'LUAD',
       'STAD', 'COAD', 'LGG', 'LUAD', 'LUAD', 'LGG', 'LGG', 'LIHC',
       'LIHC', 'LUAD', 'LGG', 'COAD', 'LGG', 'LUAD', 'LIHC', 'COAD',
       'LGG', 'STAD', 'STAD', 'LGG', 'COAD', 'LIHC', 'STAD', 'LUAD',
       'STAD', 'LIHC', 'LUAD', 'COAD', 'LUAD', 'COAD', 'COAD', 'LGG',
       'STAD', 'STAD', 'LUAD', 'LGG', 'LGG', 'LIHC', 'COAD', 'LGG',
       'STAD', 'LGG', 'LUAD', 'STAD', 'LUAD', 'STAD', 'LUAD', 'LGG',
       'LIHC', 'LUAD', 'ST

### 여기서부터는 테스트 데이터 돌리기

In [24]:
test_cna_df = pd.read_csv("./testData/test_data_cna.csv", encoding='utf-8')

test_cna_df

,Hugo_Symbol,Entrez_Gene_Id,patient_1,patient_18,patient_34,patient_35,patient_5,patient_65,patient_7,patient_8,...,patient_52,patient_53,patient_54,patient_28,patient_68,patient_57,patient_58,patient_59,patient_72,patient_61
0,ACAP3,116983.0,0.032,0.002,-0.217,-0.013,0.173,-0.006,0.019,-0.012,...,0.005,-0.002,0.003,0.036,0.000,-0.472,-0.764,-0.949,0.006,-0.713
1,ACTRT2,140625.0,0.032,0.002,-0.217,-0.013,0.173,-0.006,0.019,-0.012,...,0.005,-0.002,0.003,0.036,0.000,-0.472,-0.764,-0.949,0.006,-0.713
2,AGRN,375790.0,0.032,0.002,-0.217,-0.013,0.173,-0.006,0.019,-0.012,...,0.005,-0.002,0.003,0.036,0.000,-0.472,-0.764,-0.949,0.006,-0.713
3,ANKRD65,441869.0,0.032,0.002,-0.217,-0.013,0.173,-0.006,0.019,-0.012,...,0.005,-0.002,0.003,0.036,0.000,-0.472,-0.764,-0.949,0.006,-0.713
4,ATAD3A,55210.0,0.032,0.002,-0.217,-0.013,0.173,-0.006,0.019,-0.012,...,0.005,-0.002,0.003,0.036,0.000,-0.472,-0.764,-0.949,0.006,-0.713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25123,VBP1,7411.0,0.095,0.043,0.019,-0.118,0.102,-0.003,0.024,-0.106,...,0.042,0.014,0.003,-0.005,0.035,-0.070,-0.022,-0.108,-0.045,-0.024
25124,DDX11L16|chrX,727856.0,0.095,0.043,0.019,-0.118,0.102,-0.003,0.024,-0.106,...,0.042,0.014,0.003,-0.005,0.035,-0.070,-0.022,-0.108,-0.045,-0.024
25125,IL9R|chrX,3581.0,0.095,0.043,0.019,-0.118,0.102,-0.003,0.024,-0.106,...,0.042,0.014,0.003,-0.005,0.035,-0.070,-0.022,-0.108,-0.045,-0.024
25126,SPRY3|chrX,10251.0,0.095,0.043,0.019,-0.118,0.102,-0.003,0.024,-0.106,...,0.042,0.014,0.003,-0.005,0.035,-0.070,-0.022,-0.108,-0.045,-0.024


In [25]:
test_cna_set = test_cna_df.drop(['Hugo_Symbol'],axis=1)

test_cna_set=test_cna_set.transpose()

test_cna_set = test_cna_set.rename(columns=test_cna_set.iloc[0])

test_cna_set = test_cna_set.drop(test_cna_set.index[0])
test_cna_set

,116983.0,140625.0,375790.0,441869.0,55210.0,83858.0,219293.0,54998.0,126792.0,54991.0,...,101927830.0,100132963.0,677817.0,677835.0,55217.0,7411.0,727856.0,3581.0,10251.0,6845.0
patient_1,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,...,0.095,0.095,0.095,0.095,0.095,0.095,0.095,0.095,0.095,0.095
patient_18,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,...,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043
patient_34,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,...,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019
patient_35,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,...,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118
patient_5,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,...,0.102,0.102,0.102,0.102,0.102,0.102,0.102,0.102,0.102,0.102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
patient_57,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,...,-0.086,-0.086,-0.086,-0.083,-0.083,-0.070,-0.070,-0.070,-0.070,-0.070
patient_58,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,...,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022
patient_59,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,...,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108
patient_72,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,...,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045


In [26]:
test_cna_set = test_cna_set.dropna()
test_cna_set

,116983.0,140625.0,375790.0,441869.0,55210.0,83858.0,219293.0,54998.0,126792.0,54991.0,...,101927830.0,100132963.0,677817.0,677835.0,55217.0,7411.0,727856.0,3581.0,10251.0,6845.0
patient_1,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,...,0.095,0.095,0.095,0.095,0.095,0.095,0.095,0.095,0.095,0.095
patient_18,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,...,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043
patient_34,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,-0.217,...,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019,0.019
patient_35,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,-0.013,...,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118,-0.118
patient_5,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,...,0.102,0.102,0.102,0.102,0.102,0.102,0.102,0.102,0.102,0.102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
patient_57,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,-0.472,...,-0.086,-0.086,-0.086,-0.083,-0.083,-0.070,-0.070,-0.070,-0.070,-0.070
patient_58,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,-0.764,...,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022
patient_59,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,-0.949,...,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108,-0.108
patient_72,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,...,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045


In [27]:
test_cna_set.isnull().sum()

116983.0    0
140625.0    0
375790.0    0
441869.0    0
55210.0     0
           ..
7411.0      0
727856.0    0
3581.0      0
10251.0     0
6845.0      0
Length: 25128, dtype: int64

In [28]:
#환자1부터 차례대로 있게 하는것
test_cna_set = test_cna_set.iloc[test_cna_set.index.astype(str).str.extract('(\d+)').astype(int).sort_values(by=0).index]
test_cna_set

,116983.0,140625.0,375790.0,441869.0,55210.0,83858.0,219293.0,54998.0,126792.0,54991.0,...,101927830.0,100132963.0,677817.0,677835.0,55217.0,7411.0,727856.0,3581.0,10251.0,6845.0
patient_1,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,0.032,...,0.095,0.095,0.095,0.095,0.095,0.095,0.095,0.095,0.095,0.095
patient_2,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,...,-0.244,-0.244,-0.244,0.323,0.323,0.323,0.323,0.323,0.323,0.323
patient_3,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075,-0.075,...,0.022,0.022,0.046,0.046,0.046,-0.379,-0.379,-0.379,-0.379,-0.379
patient_4,-0.584,-0.584,-0.584,-0.584,-0.584,-0.584,-0.584,-0.584,-0.584,-0.584,...,-0.100,-0.100,-0.100,-0.100,-0.100,-0.100,-0.100,-0.100,-0.100,-0.100
patient_5,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,...,0.102,0.102,0.102,0.102,0.102,0.102,0.102,0.102,0.102,0.102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
patient_70,0.120,0.120,0.120,0.120,0.120,0.120,0.120,0.120,0.120,0.120,...,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.004
patient_71,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,-0.022,...,-0.058,-0.058,-0.058,-0.058,-0.058,-0.058,-0.058,-0.058,-0.058,-0.058
patient_72,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,...,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045,-0.045
patient_73,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,...,-0.619,-0.619,-0.619,-0.619,-0.619,-0.619,-0.619,-0.619,-0.619,-0.619


In [30]:
test_pred = final_model.predict(test_cna_set)

In [31]:
test_pred

array(['STAD', 'LUAD', 'LUAD', 'LIHC', 'STAD', 'LIHC', 'STAD', 'STAD',
       'STAD', 'LUAD', 'LUAD', 'LUAD', 'LIHC', 'STAD', 'LUAD', 'COAD',
       'LUAD', 'STAD', 'LUAD', 'LUAD', 'LIHC', 'LUAD', 'LUAD', 'LIHC',
       'LUAD', 'LIHC', 'LIHC', 'LGG', 'LIHC', 'LIHC', 'LUAD', 'LIHC',
       'LIHC', 'STAD', 'STAD', 'LIHC', 'COAD', 'COAD', 'COAD', 'COAD',
       'LUAD', 'COAD', 'LGG', 'COAD', 'LUAD', 'COAD', 'COAD', 'LGG',
       'LGG', 'STAD', 'LGG', 'LGG', 'LGG', 'LGG', 'LIHC', 'LIHC', 'LGG',
       'LGG', 'LGG', 'COAD', 'LGG', 'LIHC', 'STAD', 'LIHC', 'COAD',
       'LUAD', 'STAD', 'LGG', 'LUAD', 'LGG', 'STAD', 'LGG', 'LGG', 'LUAD'],
      dtype=object)

In [32]:
data = {'id': ['patient_' + str(i) for i in range(1, len(test_pred)+1)],
        'expected': test_pred}

df = pd.DataFrame(data)
df.to_csv('cna_predictions.csv', index=False)